# Hyperstack with memmap
This notebook shows how to create an hyperstack from the ome tiff files or from a btf file.


See: 
https://forum.image.sc/t/ome-tiff-z-stack-to-bigtiff/50964/7

In [6]:
import tifffile as tiff
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import re
from imutils.src import imfunctions
import os
from natsort import natsorted
import math

### create an empty TZYX ImageJ hyperstack

In [ ]:
#create an empty TZYX ImageJ hyperstack
new_hyperstack_path='/scratch/zimmer/DanielM/hyperstack/2021-07-01_15-08-56_worm3_on-channel-1-pco_camera2bigtiff.btf'
hyperstack = tiff.memmap(
    new_hyperstack_path,
    shape=(4997, 20, 650,840),
    dtype='uint16',
    imagej=True,
    metadata={'axes': 'TZYX'},
)

print('A new hyperstack with the type ',type(hyperstack), 'has been created. It has shape: ', hyperstack.shape)


## write entire ome.tiff into hyperstack

In [ ]:
path='/scratch/zimmer/DanielM/BAG_Gcamp_07_01/2021-07-01_15-08-56_worm3_on-channel-1-pco_camera2/'


In [ ]:
c=0
z_index=0
t_index=0
for file in natsorted(os.listdir(path)):
    if file.endswith('ome.tif'):
        print(os.path.join(path,file))
        with tiff.TiffFile(os.path.join(path,file), multifile=False) as tif:
            for idx,page in enumerate(tif.pages):
                img = page.asarray()
                hyperstack[t_index, z_index] = img
                hyperstack.flush()
                c=c+1
                z_index=z_index+1
                if z_index==20:
                    z_index=0
                    t_index=t_index+1

### As a function:
I wrote now a function in the imutils package and it works:

In [ ]:
from imutils.src import imfunctions

In [ ]:
input_path='/scratch/zimmer/DanielM/BAG_Gcamp_07_01/2021-07-01_15-08-56_worm3_on-channel-1-pco_camera2/'
output_path='/scratch/zimmer/DanielM/hyperstack/function_test.btf'
shape=(4997, 20, 650,840)
dtype='uint16'
imagej=True,
metadata={'axes': 'TZYX'}

In [ ]:
imfunctions.make_hyperstack_from_ometif(input_path,output_path,shape,dtype,imagej,metadata)

### Write single plane

In [ ]:
# write an image to the numpy.memmap array
hyperstack[t_index, z_index] = image
hyperstack.flush()

# making hyperstacks by iterating through a pandas with its dimensions
since there are issuses with reding metadata form the tiff files I calculated and stored number of volumes, z number height and width of each recording in a dataframe.

## creating the dataframe

In [2]:
#getting file names
path='/scratch/zimmer/DanielM/ZIm443/flurescence_btiffs/cam1/'
for file in natsorted(os.listdir(path)):
    if file.endswith('.btf'):
        before,sep,after=file.partition('-channel')
        print(before)

2021-07-14_18-24-00_zim443_wor1_off
2021-07-15_13-26-57_zim443_wor1_off
2021-07-15_14-27-22_zim443_wor2_on
2021-07-15_15-12-30_zim443_wor3_on
2021-07-15_15-33-13_zim443_wor3_on
2021-07-15_16-49-36_zim443_wor4_on
2021-07-15_17-14-13_zim443_wor5_off
2021-07-15_17-48-38_zim443_wor6_off


In [3]:
#read in csv with filenames and z number which i got from onenote, as well as h and w which are constant
filename='/groups/zimmer/Daniel_Mitic/code/imutils/imutils/zim443_dimensions.csv'
df=pd.read_csv(filename,sep=';')

FileNotFoundError: File b'/groups/zimmer/Daniel_Mitic/code/imutils/imutils/zim443_dimensions.csv' does not exist

In [4]:
#getting the number of volumes
path='/scratch/zimmer/DanielM/ZIm443/flurescence_btiffs/cam1/'

In [7]:
#volumes=len of pages/z_stacks
#some of them are not integers: if z is an uneven number the last volume is not gonna have a full amount of stacks. this is why i round it up with math(ceil)
for i,file in enumerate(natsorted(os.listdir(path))):
    if 'camera' in file:
        with tiff.TiffFile(os.path.join(path,file)) as tif:
            print(math.ceil(len(tif.pages)/df.iloc[i,2]))

NameError: name 'df' is not defined

In [ ]:
#i inserted the number of volumes manually in the csv, so i read it in again here
filename='/groups/zimmer/Daniel_Mitic/code/imutils/imutils/zim443_dimensions.csv'
df=pd.read_csv(filename,sep=';')

# create the hyperstacks

In [ ]:
#inputs  make_hyperstack_from_ometif function
path='/scratch/zimmer/DanielM/ZIm443/cam1/' #path with the folders containing the ome.tif
output_file='/scratch/zimmer/DanielM/hyperstack_daniel/'
dtype='uint16'
imagej=True,
metadata={'axes': 'TZYX'}

In [ ]:
for i,file in enumerate(natsorted(glob.glob(path+'*camera*'))): 
    before,sep,name=file.partition('cam1/') #here i grab the name of the file
    output=os.path.join(output_file+name+'.btf') #making sure the output files have different names
    input_path=os.path.join(path+name+'/') #here i specify the input path as the subfolders which contain the ome.tifs
    shape=tuple(df.iloc[i,1:5]) #iterate through the dataframe to get the corresponding shape for each recording
    imfunctions.make_hyperstack_from_ometif(input_path,output,shape,dtype,imagej,metadata)
print('done')